In [1]:
import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import logging
import numpy as np
import blackboard
import ka
import ka_rp as karp
import ka_br as kabr
import database_generator as dg
import bb_basic
import time
import h5py

In [3]:
ind, obj = dg.get_data(['height', 'smear', 'pu_content'], ['cycle length'])
print(obj)

[[None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [None], [array(-197.29987294)], [None], [array(-407.51905752)], [array(-51.26995646)], [array(240.31755915)], [array(-291.10751819)], [array(100.78903654)], [array(415.83742116)], [array(-161.26373626)], [array(-58.87986071)], [array(45.70957096)], [array(167.43192739)], [array(-642.17939214)], [array(-256.73139159)], [array(123.76203576)], [array(243.30916282)], [array(378.08302808)], [array(-517.48747913)], [array(-99.80916031)], [array(315.01930502)], [array(442.57246377)], [array(590.86538462)], [array(-408.36538462)], [array(69.06052394)], [array(514.59265891)], [array(660.18596788)], [array(816.95379796)], [array(-284.77564103)], [array(246.71968191)], [None], [None], [None], [array(271.39917695)], [array(594.42090395)], [array(-2.19885277)], [array(481.73598553)], [array(858.75518672)], [array(193.48739496)], [array(189.80352304

In [3]:
ns = run_nameserver()
bb = run_agent(name='bb', base=bb_basic.BbTraditional)

bb.connect_agent('rp', 'ka_rp')
bb.connect_agent('br', 'ka_br')

bb.add_abstract_lvl(1, {'valid': bool})
bb.add_abstract_lvl(2, {'reactor parameters': {'height': float, 'smear': float, 'pu_content': float, 'keff': float, 'void_coeff': float, 'doppler_coeff': float}})
#bb.load_h5()

Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:16801 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:16801
INFO [2020-04-14 18:34:09.242815] (bb): Connected agent ka_rp of agent type rp
INFO [2020-04-14 18:34:09.281582] (bb): Connected agent ka_br of agent type br
NS shut down.


In [ ]:
while not bb.get_attr('_complete'):
    bb.publish_trigger()
    bb.controller()
    bb.send_executor()
    bb.wait_for_ka()
    print('Finished trigger event: {}'.format(bb.get_attr('_trigger_event')))

print(bb.get_attr('abstract_lvls')['level 1'])

In [ ]:
h5 = h5py.File('bb_archive.h5', 'r')
for k,v in h5['level 2']['core_[67.38, 57.01, 0.42]']['reactor parameters'].items():
    print(k,v[0])
h5.close()